In [ ]:
# 18.07.23 FR - Doppelte Anweisung für ein Feld (buySell) entfernt
# 17.07 23 FR - Korrektur wegen des neuen Feldes 'notes', das gestern als 'code' eingefügt wurde 
# 17.07 23 FR - Aktualisierung der SQL-Anweisung zum Füllen der Felder zwecks Angleichung an die Tabellen-Struktur in der DB
# 16.07 23 Bg - underlyingSymbol + code + symbolSort (wegen Report) ergänzt 
# 13.07.23 BG - Dateiname wird oben dargestelt 
# 11.07.23 BG - Neues Datumsformat mm-dd-yy
# 11.07.23 BG - Mit Ausgabe Dateinamen
# 11.07.23 BG Neues Feld action, aktiv selber füllen  
# 11.07.23 BG Neus Feld fxRateToBase 
# 10.07.23 BG neue Felder capitalGainsPnl / fxPnl 
# 09.07.23 BG ohne Account ID 
# 23.06.23 BG - none -> None 
# 22.06.23 Bg Ausgabe mit +-Zeichen / None eingeführt 
# 22.06.23 BG tradeDate + und dateTime als echte Datums/Uhrzeitfelder 
# 21.06.23 BG ibCommission - Feld ergänzt  
# 21.06.23 BG neue Feldnamen in das Program integriert (war alles OK) 
# 17.06.23 BG openClose in Langtext umbenannt 
# 15.06.23 BG assetCategory eergänzt (OPT) , multiplier
# 12.06.23 BG erster Wurf
#
# 
#
#verwendeter Report: Trades
#
# Infos zu dem Report 
# https://ibkrguides.com/reportingreference/reportguide/tradesfq.htm
#

dateiname = "Trades_Basiswaehrung.xml"
#dateiname = "GmbH-Trades.xml"
# dateiname = "Trades_Statements_LX_2023_v1.xml"
# dateiname = "alex-trades.xml"

#--------------------------

import locale
from xml.dom.minidom import parse
from datetime import datetime

import pyodbc  # Datenbankanbindung an Access 

# -------------  TK - Iner Anbindung (laut Chat GPT )
import tkinter as tk
from tkinter import filedialog
from tkinter import messagebox

#----------------- End of Import ----------------------

locale.setlocale(locale.LC_ALL, 'de_DE')
# locale.setlocale(locale.decimal_point, ',' )
#print (locale.localeconv())

def IBStr2Date ( dateStr):
    formatting = "%Y-%m-%d"
    date = datetime.strptime(dateStr, formatting)
    return date

def IBStr2DateTime ( dateTimeStr):
    formatting = "%Y-%m-%d %H:%M:%S"
    dateTime = datetime.strptime(dateTimeStr, formatting)
    return dateTime

# -------------  TK - Iner Anbindung (laut Chat GPT )
import tkinter as tk
from tkinter import filedialog
from tkinter import messagebox

def insertTrades ( item , dbCursor , dbConn):
    
    description = item.getAttribute ("activityDescription") 
    code = item.getAttribute ("activityCode") 
    transactionID = item.getAttribute ("transactionID") 
    tradeID = item.getAttribute ("tradeID")  
    if ( tradeID == ""):
        tradeID = 0 

    #accountID = item.getAttribute ("accountId") 
    accountID = "---"
        
    
    #dateTimeStr = item.getAttribute ("dateTime") # Nur in dem Feld ist immer ein Datum drin  
    #dateTime = IBStr2DateTime ( dateTimeStr )
    
    # Weitere Felder bei Trades 
    conid = item.getAttribute ("conid") 
    description = item.getAttribute ("description") 
    dax = description.split(' ', 1)[0]
    symbol = item.getAttribute ("symbol")   
    # symbol = symbol.split(' ', 1)[0]
    underlyingSymbol = item.getAttribute ("underlyingSymbol")   
        
    # Erklärung: 
        # Beim Kaufen ist Cost die Gesamtkosten der Anschaffung 
        # beim Verkauf sind Cost die Anteiligen Anschaffungskosten 
        # und der  fifoPnlRealized ist der Gewinn/Verlust, der durch diesen Verkauf realiert wird

    assetCategory=item.getAttribute ("assetCategory") # 15.06 Neu dazu 
    multiplier = int(item.getAttribute ("multiplier")) # 15.06 Neu dazu  
    transactionType = item.getAttribute ("transactionType")# 15.06 Neu dazu
    fifoPnlRealized = float(item.getAttribute ("fifoPnlRealized")) # 15.06. Beu - 
    # 10.07.23 Bg Ohne Float Probiert 
    #fifoPnlRealized = item.getAttribute ("fifoPnlRealized") # 15.06. Beu - 
    # 10.07.23 Bg neue Felder capitalGainsPnl / fxPnl 
 
    capitalGainsPnl = item.getAttribute ("capitalGainsPnl") # 10.07. Beu - 
    if capitalGainsPnl == "":
        capitalGainsPnl = 0.0
    else:
         capitalGainsPnl = float (capitalGainsPnl)
    fxPnl = float(item.getAttribute ("fxPnl")) # 10.07 Neu  - 
    #print ( f"Capital: {capitalGainsPnl}  pln {fxPnl}")
    
    # 11.07. - Neu : fxRateToBase
    fxRateToBase = float (item.getAttribute ("fxRateToBase")) # 11.07 Neu  - 

    if ( dax == "DAX" ):
        symbol = dax 

    quantity = item.getAttribute ("quantity")  
    cost = float(item.getAttribute ("cost"))     
    putCall = item.getAttribute ("putCall") 
    notes = item.getAttribute ("notes") 

    openClose= item.getAttribute ("openCloseIndicator") 
    if openClose == "C" :
        openClose = "Close"
    elif openClose == "O":
        openClose = "Open"
    # Buy or sell.
    buySell = item.getAttribute ("buySell")

    # 11.07.23 - eigenes Zusatzfeld füllen aus openClose und buySell 
    # BTO (Buy-to-Open), BTC (Buy-to-Close), STO (Sell-to-Open), STC (Sell-to-Close)
    if openClose == "Open":
        if buySell == "BUY":
            action = "BTO" # Buy-to-Open
        else:
            action = "STO" # Sell-to-Open
    else:      
        # Close-Zweig 
        if buySell == "BUY":  
            action = "BTC" # Buy-to-Close 
        else:
            action = "STC" # Sell-to-Close 
    
    
    strike = item.getAttribute ("strike") 

    tradePrice = float(item.getAttribute ("tradePrice")) # 11.07. Nicht mehr runden  
    result = float(item.getAttribute ("fifoPnlRealized")) 
    quantity = item.getAttribute ("quantity") 
    
    IBCurrency = item.getAttribute ("currency") # 22.06. Name geändert 
    
    expiry = item.getAttribute ("expiry") 
    if ( expiry != "" ):
        # 22.06 UMgestellt auf Funktion
        expiryDate = IBStr2Date ( expiry)       
    else:
        expiryDate = None 

    tradeDateStr = item.getAttribute ("tradeDate") 
    # 22.06 BG umgestellt auf Funktion 
    tradeDate = IBStr2Date(tradeDateStr)

    dateTimeStr = item.getAttribute ("dateTime") # 22.06. Jetzt original Datumsfeld 
    # Abweichender Name, weil DateTime ein Schlüsselwort ist 
    IBDateTime = IBStr2DateTime ( dateTimeStr )

    # Zwei Sortiervarianten zur Auswahl
    # symbolSort = f"{underlyingSymbol} {expiry} {strike} - {putCall}" # Zum leichteren Sortieren
    symbolSort = f"{underlyingSymbol} {putCall} - {expiry} {strike}" # Zum leichteren Sortieren

    ibCommission = item.getAttribute ("ibCommission")
    if ( ibCommission == "" ):
        ibCommission = 0.00
    else:
         ibCommission = float(ibCommission)
 
    # print ( "Transaktion:" , transactionID, " COnID", conid, "Beschreibung " , description , "Datum: " , tradeDatum , "Experiy", expiryDatum , "Time: ", dateTime) 
    # In die Datenbank eintragen 
    # print ( assetCategory, "-", multiplier, "-", transactionType, "-", fifoPnlRealized )

    if ( transactionID != ""):
        transactionID = int(transactionID )

        sql = "INSERT INTO trades "
        sql += "(transactionID, accountID, cost, tradeID, description, symbol, underlyingSymbol, symbolSort, IBCurrency, conid, putCall, action, buySell, openClose "
        sql += ", notes, tradePrice, ibCommission , tradeDate, IBDateTime, strike, expiryDate, quantity, assetCategory, multiplier, transactionType, fifoPnlRealized "
        sql += ", capitalGainsPnl, fxPnl, fxRateToBase"
        sql += ") "
        sql += "VALUES (?, ?, ?, ?, ?, ? "      
        sql += ", ?, ?, ?, ? , ?, ?, ?, ?, ?, ?, ?, ? ,?, ?"
        sql += ", ?, ?, ?, ?, ? , ?, ?, ?, ?"
        sql += ")"
    
        # print (sql)

        data = (transactionID, accountID, cost, tradeID, description, symbol, underlyingSymbol, symbolSort, IBCurrency, conid, putCall, action, buySell, openClose\
              , notes, tradePrice, ibCommission, tradeDate, IBDateTime, strike, expiryDate, quantity, assetCategory, multiplier, transactionType, fifoPnlRealized\
                , capitalGainsPnl, fxPnl, fxRateToBase \
            )

        dbCursor.execute(sql, data)
        dbConn.commit()

def select_file():
    global dateiname, file_label
    dateiname = filedialog.askopenfilename()
    if dateiname:
        # print("Selected file:", dateiname )
        file_label.config ( text = "Dateiname ausgewählt" + dateiname) 

def on_window_close():
    if messagebox.askokcancel("Quit", "Do you want to quit?"):
        window.destroy()
        
# --------- Start des Programms --------------------------

#Datenbank öffnen 
# Verbindung zur Access-Datenbank herstellen
dbConn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=.\ib.accdb;')

# Cursor-Objekt erstellen
dbCursor = dbConn.cursor()

#alle Daten löschen 
sql= "DELETE FROM trades"
dbCursor.execute (sql) 

i = 0 ; 
limit = 70 


# ----- Grafische Oberfläche -------- Create the main window
window = tk.Tk()
window.title("Tkinter File Select")

# Create a button to open the file dialog
button = tk.Button(window, text="Datei auswählen", command=select_file) ; button.pack()
file_label = tk.Label(window, text="No file selected"); file_label.pack()
window.protocol("WM_DELETE_WINDOW", on_window_close)
# Run the main event loop
# window.mainloop()
# --- Ende der grafischen Oberfläche -----------------


print (f"Start mit {dateiname}")

elementname = 'Trade'
domobject = parse (dateiname)

for e in domobject.getElementsByTagName( elementname ):
    #Übertragen der Daten in die Datenbank 
    insertTrades ( e , dbCursor, dbConn)
    
    i = i+1
    if i % limit == 0  :
        print ("")
        print (i, end="")
    print("+", end="")

print ("")
print ( "Anzahl Datensätze eingetragen :" , i ) 
    
# Änderungen in der Datenbank speichern
dbConn.commit()
# Verbindung schließen
dbConn.close()

print (f"Fertig mit {dateiname}")
    
